<a href="https://colab.research.google.com/github/Persie0/Online-Digi4school-PDF-Downloader/blob/main/Digi4school.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gib unten alles ein und drücke Strg+F9 oder chronologisch immer auf den Button mit dem Pfeil
(Maus über den Code plazieren, dann erscheint Button)

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
!pip install --upgrade pip
!pip install PyPDF2
!pip install svglib
!pip install reportlab



In [ ]:
#@title PDF Download
#@markdown zB https://a.digi4school.at/ebook/6320/
ebook_url = "" #@param {type:"string"}
#@markdown und

#email = "" #@param {type:"string"}
#passwort = "" #@param {type:"string"}
#@markdown Cookies finden: https://github.com/Persie0/Online-Digi4school-PDF-Downloader/raw/main/2022-12-10_13-20.png im browser gewünschtes buch öffnen, dann Rechtsklick und Element untersuchen - anschließend unter Webspeicher "Cookies" suchen, anschließend ohne Anführungszeichen eingeben zB 22365898844%2c0%2c0%2c167688940033%20{226%201670641233%208989870A0DC07062E5F6AFFDS86561090B3126830}
digi4s_cookie = "" #@param {type:"string"}
digi4b_cookie = "" #@param {type:"string"}

from svglib.svglib import svg2rlg
from reportlab.graphics import renderPDF, renderPM
from svglib.svglib import SvgRenderer
from reportlab.graphics import renderPM
from bs4 import BeautifulSoup
import requests
from google.colab import output
import sys
import os
import time
from PyPDF2 import PdfFileMerger

# Replace this with the cookie you want to send with the request
# The cookie should be a dictionary containing the cookie's name and value
cookie = {"digi4s": digi4s_cookie,"digi4b": digi4b_cookie}
headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36"
}

"""
# Check the response status code to make sure the login was successful
if email!="" and passwort!="":
  # Set the URL of the login page
  url = "https://digi4school.at/br/xhr/login"


  # Set the login credentials
  payload = {
      "email": email,
      "password": passwort
  }

  # Send a POST request to the login page with the credentials
  response = requests.post(url, data=payload, headers=headers)

  if response.status_code == 200:
      print("Login successful!")
      # Get the cookies from the response
      global cookies
      cookies = response.cookies
  else:
      print("Login failed.")
"""

# Replace this with the names of the SVG images you want to convert
image_names = []


response = requests.get(ebook_url, cookies=cookie)
soup = BeautifulSoup(response.text, 'html.parser')
title_tag = soup.find('meta', {'name': 'title'})
title = title_tag['content']
for page in range(50000):
  strpage=str(page+1)
  # Replace this with the URL of the website you want to scrape
  url = ebook_url+strpage+"/"+strpage+".svg"

  # Send the request and get the page as a string
  response = requests.get(url, cookies=cookie)
  output.clear()
  print(title,"\nAm Downloaden auf Seite "+strpage)
  time.sleep(1.5)


  # Check the response status code to make sure the request was successful
  if response.status_code == 200:
      image_names.append(strpage+".svg")
      # Save the SVG image file to your computer
      with open(strpage+".svg", "wb") as f:
          f.write(response.content)
  else:
    strpage=str(page+1)
  # Replace this with the URL of the website you want to scrape
    url = ebook_url+strpage+".svg"

  # Send the request and get the page as a string
    response = requests.get(url, cookies=cookie)
    time.sleep(2)

  # Check the response status code to make sure the request was successful
    if response.status_code == 200:
        image_names.append(strpage+".svg")
        # Save the SVG image file to your computer
        with open(strpage+".svg", "wb") as f:
            f.write(response.content)
    else:
      print("Am Konvertieren")
      break
      

#create png from svg
for image in image_names:
  output.clear()
  print(title,"\nKonvertiere "+image)
  drawing = svg2rlg(image)
  renderPDF.drawToFile(drawing, image.replace(".svg",".pdf"))

# Create an instance of the PdfFileMerger class
merger = PdfFileMerger()

# Iterate over the PDF files
for image in image_names:
  # Open the PDF file
  output.clear()
  print(title,"\nAm PDF erstellen: "+image)
  with open(image.replace(".svg",".pdf"), "rb") as f:
    # Add the PDF file to the merger
    merger.append(f)

# Save the merged PDF file
with open("/content/drive/MyDrive/"+title+".pdf", "wb") as f:
  merger.write(f)

#delete png
for image in image_names:
  os.remove(image.replace(".svg",".pdf"))

#delete svg
for image in image_names:
  os.remove(image)

print("Fertig!!!")

print("auf deinem Drive als: "+title+".pdf")